In [1]:
import glob
import pickle
from tqdm import tqdm

from sqlalchemy import Column, Integer, String, Date, Text, ARRAY, FLOAT

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

from db_utils import WapoQueries, WapoDocEmbeddings, gen_session, refresh_db, get_wapo_doc2query, get_wapo_emb

In [2]:
! createdb -h postgres doc2queries

In [3]:
server_address = 'postgresql://root@postgres:5432/'

In [4]:
session = gen_session(server_address=server_address, db_name="doc2queries")

In [5]:
Base = declarative_base()

/tmp/ipykernel_10930/4196137762.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [6]:
#define tables
class WapoDoc2Qeury(Base):
    __tablename__ = 'wapo_doc2queries'
    docno = Column(String, primary_key=True)
    querygen = Column(ARRAY(Text))

    def __repr__(self):
        repr_str = f"docno={self.docno}, query={self.querygen}"
        return repr_str

class NytDoc2Qeury(Base):
    __tablename__ = 'nyt_doc2queries'
    docno = Column(String, primary_key=True)
    querygen = Column(ARRAY(Text))

    def __repr__(self):
        repr_str = f"docno={self.docno}, query={self.querygen}"
        return repr_str

In [7]:
#create tables

engine = create_engine(server_address + "doc2queries")
Base.metadata.create_all(engine)

In [8]:
d2q_paths_nyt = sorted(glob.glob("./workspace/data/d2qs_clean/nyt/*"))
d2q_paths_wapo = sorted(glob.glob("./workspace/data/d2qs_clean/wapo/*"))

In [9]:
#fill tables with doc2queries

with tqdm(total = len(d2q_paths_nyt)) as pbar:
    for path in d2q_paths_nyt:
        current_dic = pickle.load(open(path, "rb"))

        for key, val in current_dic.items():
            
            session.add(NytDoc2Qeury(docno=key, querygen=val))

        session.commit()
        pbar.update()

    session.close()

100%|██████████| 103/103 [02:23<00:00,  1.40s/it]


In [10]:
with tqdm(total = len(d2q_paths_wapo)) as pbar:
    for path in d2q_paths_wapo:
        current_dic = pickle.load(open(path, "rb"))

        for key, val in current_dic.items():
            
            session.add(WapoDoc2Qeury(docno=key, querygen=val))

        session.commit()
        pbar.update()

    session.close()

100%|██████████| 101/101 [00:44<00:00,  2.26it/s]
